In [22]:
#Import libraries
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import string
import nltk
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [23]:
#Import the dataset
df = pd.read_csv("transcriptions_2.csv") #transcriptions
df1=pd.read_csv("all_commercials_classified_filtered.csv") # Nice_categories

In [24]:
#Join the two datasets
df_total=pd.concat([df.set_index('commercial_id'),df1.set_index('commercial_id')], axis=1, join='inner') #Join on commercial_id
#Keep only the transcription with log_prob >=-0.5
df_total=df_total.loc[df_total['log_prob']>=-0.5]

#Remove all the Nan values of nice_class, they are marked by -1
df_total=df_total.loc[df_total['nice_class']>-1]

#Rimuovo i casi in cui la classe di nizza ha gruppi molto piccoli (altrimenti lo split non funziona)
class_counts = df_total['nice_class'].value_counts()
valid_classes = class_counts[class_counts >= 4].index
df_total= df_total[df_total['nice_class'].isin(valid_classes)]

#Select only the columns of interest: transcriptions (of the descriptions) and nice_class
df_total.drop('title', axis=1, inplace=True)
df_total.drop('log_prob', axis=1, inplace=True)

df_total.head()

,transcription,nice_class,description,year,lustrum,duration_in_seconds,source,has_been_transcribed,palette_extracted
commercial_id,,,,,,,,,
_3siiuQ3kp0.s5,Nascere nuovo Fiorino Franco e Hugo stabilisco...,12,NaN,1993,1990_1994,31.0,mDeplo (YouTube),True,True
_LQYpDfplFE,"Caldo, benvenuto a tutti da questo fantastico ...",25,Vecchia pubblicità della Nike trovata su una V...,2004,2000_2004,60.0,Stefano Tamburini (YouTube),True,True
_qPReXLa870,La bottiglia Coca Cola con Piacento Alli. Chie...,32,Pubblicità Coca Cola spot 2015 anniversario,2015,2015_2019,30.0,SuperSpot (YouTube),True,True
_RIBAprIKQc,Sottotitoli creati dalla comunità Amara.org,14,Morellato in questa stagione celebra l’amore! ...,2014,2010_2014,16.0,Morellato (YouTube),True,True
_VxI8Bo64wQ.s5,"non c'è, non c'è ciao, o ci stai parlando? e.....",38,NaN,2000,2000_2004,31.0,mDeplo (YouTube),True,True


In [25]:
df_filtered=df_total[['transcription', 'nice_class']].copy()

In [26]:
#Remove the punctuation from the transcriptions
def remove_punctuation(text):
    if pd.isna(text):
        return text
    punctuationfree="".join([i for i in str(text) if i not in string.punctuation])
    return punctuationfree

df_filtered['transcription'] = df_filtered['transcription'].apply(remove_punctuation)
#Apply the lower method to strings

df_filtered['transcription'] = df_filtered['transcription'].apply(lambda s: s.lower() if type(s) == str else s)

#Remove the stopwords
nltk.download('stopwords')
stopwords = stopwords.words('italian')
df_filtered['transcription'] = df_filtered['transcription'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stopwords)]))

df_filtered= df_filtered.reset_index(drop=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/denisebotrini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
df_filtered.head()

,transcription,nice_class
0,nascere nuovo fiorino franco hugo stabiliscono...,12
1,caldo benvenuto fantastico stadio cominciare l...,25
2,bottiglia coca cola piacento alli chiedila bar...,32
3,sottotitoli creati comunità amaraorg,14
4,cè cè ciao parlando parlando ascoltavo messagg...,38


In [28]:
X=df_filtered['transcription'].tolist()
y=df_filtered['nice_class']

In [29]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, stratify=y_train_val, random_state=0)

In [30]:
np.unique(y_train)

array([ 3,  5,  9, 12, 14, 16, 25, 28, 29, 30, 32, 33, 35, 36, 38, 43])

In [31]:
#Dopo vari tentativi, il modello migliore risulta essere un LinearSVC

In [32]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=0)
x_train = np.array(X_train)
y_train = np.array(y_train)

In [33]:
possible_C_values = [0.5, 1]
best_acc = -1
best_model = None
for C in possible_C_values:
    pipeline = Pipeline([
        ('vect', CountVectorizer()),  # feature extraction
        ('sel', SelectKBest(chi2, k=1500)),  # feature selection
        ('tfidf', TfidfTransformer()),  # weighting
        ('learner', LinearSVC(C=C))  # learning algorithm
    ])
    k_fold_acc = []
    for t_index, test_index in kf.split(x_train):
        x_train_sub, x_val = x_train[t_index], x_train[test_index]
        y_train_sub, y_val = y_train[t_index], y_train[test_index]
        model = pipeline.fit(x_train_sub,y_train_sub)
        predictions = model.predict(x_val)
        acc = accuracy_score(y_val, predictions)
        k_fold_acc.append(acc)
    acc = np.mean(k_fold_acc)
    if acc > best_acc:
        best_acc = acc
        best_model = model
    print(f"Accuracy on the validation with C {C}: {acc}")

Accuracy on the validation with C 0.5: 0.3111111111111111
Accuracy on the validation with C 1: 0.3388888888888889


/Users/denisebotrini/anaconda3/envs/env_master/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/denisebotrini/anaconda3/envs/env_master/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/denisebotrini/anaconda3/envs/env_master/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/denisebotrini/anaconda3/envs/env_master/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set t

In [34]:
test_accuracy = accuracy_score(y_test, best_model.predict(X_test))
test_accuracy

0.32432432432432434

In [35]:
best_model.predict(['ed ecco qui il vostro bellissimo bambino, è tutta questione di giorni, davvero? Mangi le doritos? Mangia le doritos durante la mia ecografia, la puoi smettere?'])

array([38])

In [36]:
best_model.predict(['ok, ho pochi secondi per convincerti a passare a ho. Copriamo il 98% del territorio e è anche il 5g, fai tutto con app'])

array([12])

In [37]:
best_model.predict(['questa è una scatola chiusa di tonno. ci dice che il suo contenuto è tonno, olio di oliva, sale. Cioè che tonno rio mare è solo tonno di ottima scelta con ottimo olio di oliva'])

array([29])

In [38]:
best_model.predict(['pippo e la fotografia lalalalala pippo è qui  un bambino ha bisogno di me per mille codini vi saluto bambini e tanti ossequi alla signora maestra, bambino tu bisogna pannolini. i pannolini più venduti in Italia presentano una straordinaria novità, pannolini per la notte. ora vado a nanna, mi faccio una dormita che dura tutta la notte']) 
#pubblicità anni 60

array([38])

In [39]:
best_model.predict(['pannolino']) 

array([12])